In [0]:
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 69.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "suyashmejari"
os.environ["KAGGLE_KEY"] = "KGAT_31685b2c29a6ef311040b4a96fd3915d"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

DataFrame[]

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

DataFrame[]

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data && kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors


100%|██████████| 4.29G/4.29G [00:57<00:00, 80.1MB/s]


In [0]:
%sh 
cd /Volumes/workspace/ecommerce/ecommerce_data && unzip -o ecommerce-behavior-data-from-multi-category-store.zip && ls -lh


Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-f79592cd-f592-4989-8dff-63 nogroup 8.4G Jan  9 19:19 2019-Nov.csv
-rwxrwxrwx 1 spark-f79592cd-f592-4989-8dff-63 nogroup 5.3G Jan  9 19:22 2019-Oct.csv
-rwxrwxrwx 1 spark-f79592cd-f592-4989-8dff-63 nogroup 4.3G Jan  9 19:17 ecommerce-behavior-data-from-multi-category-store.zip


In [0]:
%sh
%restart_python 

bash: line 1: fg: no job control


In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

In [0]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()

October 2019 - Total Events: 42,448,765

SCHEMA:
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
df.show(5, truncate=False)


SAMPLE DATA (First 5 rows):
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+------+---------+------------------------------------+
|_c0                    |_c1       |_c2       |_c3                |_c4                                |_c5     |_c6   |_c7      |_c8                                 |
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                      |brand   |price |user_id  |user_session                        |
|2019-10-01 00:00:00 UTC|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79 |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00 UTC|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.20 |554748717|9333dfb

In [0]:
# Load your data
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

# Verify it's working
print(f"✅ Ready to go! Loaded {events.count():,} events")
events.show(5)
data = [("iPhone", 999), ("Samsung", 799), ("MacBook", 1299)]
df = spark.createDataFrame(data, ["product", "price"])
df.show()

# Filter expensive products
df.filter(df.price > 1000).show()

✅ Ready to go! Loaded 42,448,765 events
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|                 _c0|       _c1|       _c2|                _c3|                 _c4|     _c5|   _c6|      _c7|                 _c8|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                NULL|shiseido| 35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua| 33.20|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|543.10|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:...|      v